In [1]:
from nbutils import style
style()

<div class="nbhero">
    <h1 class="nb title">Random Forest Classification</h1>
    <h2 class="nb subtitle">Single Node no GPU</h2>
    <div class="nb description">
        Train a random forest on the CPU.  It's going to be slow!  You'll thank me later.
    </div>
</div>


This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll use `pandas` to load a subset of the data and `scikit-learn` to answer this classification question:

> based on characteristics that can be known at the beginning of a trip, will this trip result in a high tip?

## Monitoring Resource Usage

As you work through this tutorial, you may want to monitor the code's utilization of the available CPU and memory, to get a better understanding of how expensive different operations are.

<details><summary>(click here to learn how to monitor resource utilization)</summary>

<br>

**Monitoring CPU and Main Memory**

To monitor CPU utilization and the amount of free main memory, you can use `htop`.

Open a new terminal and run `htop`. That will keep an auto-updating dashboard up that shows the CPU utilization and memory usage.

**Monitoring GPU and GPU memory**
    
Open a new terminal and run the following command.

```shell
watch -n 5 nvidia-smi
```

<br>
 
This command will update the output in the terminal every 5 seconds. It shows some information like:

* current CUDA version
* NVIDIA driver version
* internal temperature
* current utilization of GPU memory
* list of processes (if any) currently running on the GPU, and how much GPU memory they're consuming

You should see that `scikit-learn` never uses the available GPU.    

<br>

Whichever option you choose, leave these terminals with the monitoring process running while you work, so you can see how the code below uses the available resources.

</details>

<hr>

## Load data

This example is designed to run quickly with small, relatively inexpensive resources. So let's just load a single month of taxi data for training.

In [ ]:
import pandas as pd

taxi = pd.read_csv(
    "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
)

The code below computes the size of this dataset in memory. One month is about 7.6 million rows and 1.5 GB.

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {round(taxi.memory_usage(deep=True).sum() / 1e9, 2)} GB")

You can examine the structure of the data with `pandas` commands:

`.head()` = view the first few rows

In [3]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


`.dtypes` = list all the columns and the type of data in them

In [4]:
taxi.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

<hr>

## Prep for Training

Before training a model, we need to transform this dataset into a format that's better-suited to the research question. The function below does that with `pandas` operations.

<details><summary>(click here to learn why data scientists do this)</summary>

**Compute the Target**

The raw data don't contain a column that cleanly describes whether or not a particular ride could be considered a "high" tip. So we need to add one! In this exercise, a "high" tip is one that is greater than 20% of the fare amount.

**Add Features**

Giving a machine learning model a richer description of each training observation improves its ability to describe the relationship between those observations' characteristics and the target. These characteristics are called "features".

For example, instead of giving a model a raw timestamp, it can be valuable to provide multiple derived characteristics like hour of the day and day of the week. It's plausible, for example, that weekend rides might have a different distribution of tips because they tend to be for leisure, where weekday rides might be mostly people travelling for work.

**Remove Unused Features**

If the goal is to produce a model that could predict whether or not a tip will be high *at the beginning of the trip*, then characteristics that can only be known AFTER the tip have to be excluded. For example, you can't know the dropoff time or the type of payment until a ride has concluded.

Such features should be dropped before training.
    
</details>

In [5]:
def prep_df(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    """
    Prepare a raw taxi dataframe for training.
        * computes the target ('high_tip')
        * adds features
        * removes unused features
    """
    df = df.copy()
    numeric_feat = [
        "pickup_weekday",
        "pickup_hour",
        "pickup_week_hour",
        "pickup_minute",
        "passenger_count",
    ]
    categorical_feat = [
        "PULocationID",
        "DOLocationID",
    ]
    features = numeric_feat + categorical_feat

    # add target
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df["tip_fraction"] = df.tip_amount / df.fare_amount
    df[target_col] = df["tip_fraction"] > 0.2

    # add features
    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute

    # drop unused columns
    df = df[features + [target_col]].astype(float).fillna(-1)

    return df

Run the code below to get a new data frame, `taxi_train`, that can be used directly for model training.

In [6]:
target_col = "high_tip"
taxi_train = prep_df(df=taxi, target_col=target_col)

`taxi_train` is a `pandas` dataframe that will be passed in to a machine learning model. Since this is a binary classification task, before proceeding we should examine the distributions of 1s and 0s in the target. This can be done with the `.value_counts()` method.

<details><summary>(click here to learn why data scientists do this)</summary>

**Target Distribution**
    
In binary classification tasks, we ask machine learning models to learn the difference between records that produced a 0 for some target and those that produce a 1. In this example:
    
* `high_tip = 1`: "this trip resulted in a high tip"
* `high_tip = 0`: "this trip did not result in a high tip"
    
The "distribution" of this target just means "what % of the trips ended in a high tip?". This is important to understand because we might have to change our modelling approach if the distribution was uneven. For example, if 99% of trips did NOT end in a high tip, a model that just always guessed "not a high tip" would achieve an accuracy of 99%.

</details>

In [7]:
taxi_train[target_col].value_counts()

1.0    4001740
0.0    3656495
Name: high_tip, dtype: int64

Before going further, check the first few rows of the dataset to make sure that the features look reasonable.

In [8]:
taxi_train.head()

,pickup_weekday,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,PULocationID,DOLocationID,high_tip
0,1.0,0.0,24.0,46.0,1.0,151.0,239.0,1.0
1,1.0,0.0,24.0,59.0,1.0,239.0,246.0,0.0
2,4.0,13.0,109.0,48.0,3.0,236.0,236.0,0.0
3,2.0,15.0,63.0,52.0,5.0,193.0,193.0,0.0
4,2.0,15.0,63.0,56.0,5.0,193.0,193.0,0.0


Now that the dataframe has been processed, check its size in memory again.

In [9]:
print(
    f"Num rows: {len(taxi_train)}, Size: {round(taxi_train.memory_usage(deep=True).sum() / 1e9, 2)} GB"
)

Num rows: 7658235, Size: 0.55 GB


As you can see above, removing unused columns dropped the size of the training data to 0.55 GB, about one third the size of the raw data.

<div class="nbhero">
    <h1 class="nb title">Train the Model</h1>
    <h2 class="nb subtitle">The call to <i>fit</i> takes 5 minutes</h2>
    <div class="nb description">
        Checkout <a class="nb link" href="./02-rf-rapids-dask.ipynb">the same model with RAPIDS and Dask</a> while you wait.
    </div>
</div>


Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `RandomForestClassifier` from `scikit-learn`. If you've never used a random forest or need a refresher, consult ["Forests of randomized trees"](https://scikit-learn.org/stable/modules/ensemble.html#forest) in the `sciki-learn` documentation.

The code below initializes a random forest classifier with the following parameter values.

* `n_estimators=100` = create a 100-tree forest
* `max_depth=10` = stop growing a tree once it contains a leaf node that is 10 levels below the root
* `random_state=42` = ensure that every run produces the same results. The choice of `42` is irrelevant and this could be any integer.
* `n_jobs=-1` = use all available cores on this machine to train models. Note that `scikit-learn` does not use the GPU, and this only refers to CPU cores.

All other parameters use the defaults from `RandomForestClassifier`.

<details><summary>(click here to learn why data scientists do this)</summary>

**Setting max_depth**
    
Tree-based models split the training data into smaller and smaller groups, to try to group together records with similar values of the target. A tree can be thought of as a collection of rules like `pickup_hour greater than 11` and `pickup_minute less than 31.0`. As you add more rules, those groups (called "leaf nodes") get smaller. In an extreme example, a model could create a tree with enough rules to place each record in the training data into its own group. That would probably take a lot of rules, and would be referred to as a "deep" tree.
    
Deep trees are problematic because their descriptions of the world are too specific to be useful on new data. Imagine training a classification model to predict whether or not visitors to a theme park will ride a particular rollercoaster. You could measure the time down to the millisecond that every guest's ticket is scanned at the entrance, and a model might learn a rule like *"if the guest has been to the park before and if the guest is older than 40 and younger than 41, and if the guest is staying at Hotel A and if the guest enters the park after 1:00:17.456 and if the guest enters the park earlier than 1:00:17.995, they will ride the rollercoaster"*. This is very very unlikely to ever match any future visitors, and if it does it's unlikely that this prediction will be very good unless you have some reason to believe that a visitor arriving at 1:00:18 instead of 1:00:17 really changes the probability that they'll ride that rollercoaster.
    
To prevent this situation (called "overfitting"), most tree-based machine learning algorithms accept parameters that control how deep the trees can get. `max_depth` is common, and says "don't create a rule more complex than this". In the example above, that rule has a depth of 7.
    
1. visiting the park
2. has been to the park before?
3. older than 40?
4. younger than 41?
5. staying at Hotel A?
6. entered the park after 1:00:17.456?
7. entered the park before 1:00:17.995?
    
Setting `max_depth = 5` would have prevented those weirdly-specific timing rules from ever being generated.
    
Choosing good values for this parameter is part art, part science, and is outside the scope of this tutorial.
    
**Setting n_jobs=-1**

In the random forest algorithm, each tree in the forest is completely independent of the others. This means that you can cut the training time by 50% by training two trees at the same time, 75% by training 4 trees at a time, and so on. Your computer's ability to parallelize work like this is determined by the number of available CPU cores. Setting `n_jobs=-1` says "use all available CPU cores to train this model", which will result in the fastest possible training time.

The value `-1` is preferred to directly hard-coding a number, because it means that if you run this code on a different-sized machine, it will take advantage of all of that machine's resources without requiring you to remember to change the code.

</details>

In [3]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

With the classifier created, fit it to some data! The code below uses `%%time` to print out a timing, so you can see how long it takes to train. This can be used to compare `scikit-learn` to methods explored in other notebooks, or to test how changing some parameters to `RandomForestClassifier` changes the runtime for training.

**NOTE:** This will take a few minutes to run. Consider opening [this Dask + RAPIDS notebook](./rf-rapids.ipynb) and running that while you're waiting for this model to train.

In [ ]:
%%time

features = [c for c in taxi_train.columns if c != target_col]

_ = rfc.fit(taxi_train[features], taxi_train[target_col])

<hr>

## Save model

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/random_forest_scikit.pkl", "wb") as f:
    cloudpickle.dump(rfc, f)

<hr>

## Calculate metrics on test set

Machine learning training tries to create a model which can produce useful results on new data that it didn't see during training. To test how well we've accomplished that in this example, read in another month of taxi data.

In [ ]:
taxi_test = pd.read_csv(
    "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
)

Before creating predictions on this new dataset, it has to be transformed in exactly the way that the original training data were prepared. Thankfully you've already wrapped that transformation logic in a function!

In [ ]:
taxi_test = prep_df(taxi_test, target_col=target_col)

`scikit-learn` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see ["Metrics and scoring"](https://scikit-learn.org/stable/modules/model_evaluation.html) in the `sciki-learn` docs.

This tutorial uses the `roc_auc_score` to evaluate the model. This metric measures the area under the [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve. Values closer to 1.0 are desirable.

In [ ]:
from sklearn.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[:, 1]
roc_auc_score(taxi_test[target_col], preds)

<hr>

## Next Steps

In this tutorial, you learned how to train a model for a binary classification task, using `scikit-learn`. Training took 9 minutes, for a dataset that was only 0.55 GB in memory.

You can train the same model much more quickly by taking advantage of the available GPU on this machine. Try [this Dask + RAPIDS notebook](./rf-rapids-dask.ipynb) next to see how this is done.

<hr>